# Identifying Ethnicity in OpenSAFELY-TPP
This short report describes how ethnicity can be identified in the OpenSAFELY-TPP database, and the strengths and weaknesses of the methods. Ethnicity is known to be an important determinant of health outcomes, particularly during the COVID-19 outbreak where a complex interplay of social and biological factors resulted in increased exposure, reduced protection, and increased severity of illness. The recording of patients’ ethnic group in primary care can support efforts to achieve equity in service provision and outcomes. This is a living document that will be updated to reflect changes to the OpenSAFELY-TPP database and the patient records within.

## OpenSAFELY
OpenSAFELY is an analytics platform for conducting analyses on Electronic Health Records inside the secure environment where the records are held. This has multiple benefits: 

* We don't transport large volumes of potentially disclosive pseudonymised patient data outside of the secure environments for analysis
* Analyses can run in near real-time as records are ready for analysis as soon as they appear in the secure environment
* All infrastructure and analysis code is stored in GitHub repositories, which are open for security review, scientific review, and re-use

A key feature of OpenSAFELY is the use of study definitions, which are formal specifications of the datasets to be generated from the OpenSAFELY database. This takes care of much of the complex EHR data wrangling required to create a dataset in an analysis-ready format. It also creates a library of standardised and validated variable definitions that can be deployed consistently across multiple projects. 

The purpose of this report is to describe the main variables that relate ethnicity, and their relative strengths and weaknesses.

## Available Records
OpenSAFELY-TPP runs inside TPP’s data centre which contains the primary care records for all patients registered at practices using TPP’s SystmOne Clinical Information System. This data centre also imports external datasets from other sources, including A&E attendances and hospital admissions from NHS Digital’s Secondary Use Service, and death registrations from the ONS. More information on available data sources can be found within the [OpenSAFELY documentation](https://docs.opensafely.org/data-sources/intro/). 

#Methods

In OpenSAFELY-TPP, there is no categorical “ethnicity” variable to record this information. Rather, ethnicity is recorded using clinical codes, like any other clinical or administrative event, with specific codes relating to specific ethnic groups

We define three codelists to capture primary care ethnicity in OpenSAFELY-TPP : "[2020-CTV3](https://www.opencodelists.org/codelist/opensafely/ethnicity/2020-04-27)", "[2022-SNOMED](https://www.opencodelists.org/codelist/opensafely/ethnicity-snomed-0removed/2e641f61/)" and "[2021-PRIMIS](https://www.opencodelists.org/codelist/primis-covid19-vacc-uptake/eth2001/v1/)".


To evaluate how well each of these codelists are populated, we count the number of patients with at least one instance each codelist, as well as the grouping of the ethncitity themselves.

We examine trends across the whole population and by each of the following demographic and clinical subgroups to detect any inequalities.

Demographic covariates:

Age band
Sex
Ethnicity
Region
IMD
Clinical covariates:

Dementia
Diabetes
Learning disability

Any counts below 6 were redacted, and all other values were rounded to the nearest 5.

In [34]:
import sys

In [26]:
import os
import pandas as pd
import numpy as np
from itertools import product
from IPython.display import display, Markdown, Image

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.options.mode.chained_assignment = None 
pd.options.display.float_format = '{:,.0f}'.format

In [27]:

def local_patient_counts(
    definitions, output_path, code_dict="", categories=False, missing=False,
):
    import pandas as pd

    suffix = "_filled"
    overlap = "all_filled"
    if missing == True:
        suffix = "_missing"
        overlap = "all_missing"
    if categories:
        df_population = pd.read_csv(
            f"../output/{output_path}/simple_patient_counts_fullset.csv"
        ).set_index(["group", "subgroup"])
        

        df_append = pd.read_csv(
            f"../output/{output_path}/simple_patient_counts_categories_fullset.csv"
        ).set_index(["group", "subgroup"])
        df_append.drop("population", inplace=True, axis=1)
        df_append["population"] = df_population["ethnicity_new_5_filled"]
        # ensure definitions[n] in code_dict[definitions[n]] below refers to one of the definitions of interest
        definitions = [
            f"{category}_{definition}"
            for category, definition in product(
                code_dict[definitions[1]].values(), definitions
            )
        ]

    else:
        df_append = pd.read_csv(
            f"../output/{output_path}/simple_patient_counts_fullset.csv"
        ).set_index(["group", "subgroup"])

    for definition in definitions:
        if missing:
            df_append[definition + suffix] = (
                df_append["population"] - df_append[definition + "_filled"]
            )    
        df_append[definition + "_pct"] = round(
            (df_append[definition + suffix].div(df_append["population"])) * 100, 1
        )
        df_append[overlap + "_pct"] = round(
            (df_append[overlap].div(df_append["population"])) * 100, 1
        )

        # Combine count and percentage columns
        df_append[definition] = (
            df_append[definition + suffix].apply(lambda x: "{:,.0f}".format(x))
            + " ("
            + df_append[definition + "_pct"].astype(str)
            + ")"
        )
        df_append = df_append.drop(columns=[definition + suffix, definition + "_pct"])
    df_append[overlap] = (
        df_append[overlap].apply(lambda x: "{:,.0f}".format(x))
        + " ("
        + df_append[overlap + "_pct"].astype(str)
        + ")"
    )
    df_append = df_append.drop(columns=[overlap + "_pct"])
    df_patient_counts = df_append[definitions + [overlap] + ["population"]]
    # Final redaction step
    df_patient_counts = df_patient_counts.replace(np.nan, "-")
    df_patient_counts = df_patient_counts.replace("nan (nan)", "- (-)")
    df_patient_counts.columns = df_patient_counts.columns.str.replace("_", " ")
    
    display(df_patient_counts)
    if categories:
        df_patient_counts.to_csv(
                f"../output/{output_path}/local_patient_counts_categories_fullset.csv"
            )


In [28]:
### CONFIGURE ###
definitions = ['ethnicity_5', 'ethnicity_new_5', 'ethnicity_primis_5']
covariates = ['_age_band','_sex','_region','_imd','_dementia','_diabetes','_hypertension','_learning_disability']
output_path = 'from_jobserver/release_2022_07_28/output/simplified_output/5_group/tables'
suffixes = ['','_missing']

code_dict = {
    "imd": {
        0: "Unknown",
        1: "1 Most deprived",
        2: "2",
        3: "3",
        4: "4",
        5: "5 Least deprived",
    },
    "ethnicity_5": {1: "White", 2: "Mixed", 3: "Asian", 4: "Black", 5: "Other"},
    "ethnicity_new_5": {1: "White", 2: "Mixed", 3: "Asian", 4: "Black", 5: "Other"},
    "ethnicity_primis_5": {1: "White", 2: "Mixed", 3: "Asian", 4: "Black", 5: "Other"},
}



## Results

### Count of Patients

Around 14.8 million patients who have been registered in OpenSAFELY-TPP have each have all three codelists. 2020-CTV3 is the most well-populated with 19 million patients having at least one 2020-CTV3 recording of ethnicity.  

In [29]:
local_patient_counts(
         definitions,  output_path
    )

ethnicity 5  \
group               subgroup                                      
all                 with records              24,817,085 (75.2)   
age_band            0-19                       4,006,955 (63.0)   
                    20-29                      3,065,155 (72.7)   
                    30-39                      4,014,055 (81.8)   
                    40-49                      3,282,495 (82.0)   
                    50-59                      3,215,505 (80.3)   
                    60-69                      2,617,490 (80.4)   
                    70-79                      2,258,985 (79.7)   
                    80+                        2,329,560 (69.3)   
                    missing                       26,885 (31.5)   
sex                 F                         12,741,390 (76.8)   
                    M                         12,075,695 (73.5)   
region              East                       4,378,705 (76.1)   
                    East Midlands              3,301,345 (77.2)   
                    London                     1,445,455 (80.5)   
                    North East                   901,895 (76.8)   
                    North West                 1,651,170 (76.8)   
                    South East                 1,232,315 (75.0)   
                    South West                 2,529,865 (74.3)   
                    West Midlands                819,650 (81.1)   
                    Yorkshire and The Humber   2,715,235 (76.3)   
imd                 1 Most deprived            4,988,090 (76.7)   
                    2                          4,941,775 (76.2)   
                    3                          5,105,055 (75.8)   
                    4                          4,777,850 (74.8)   
                    5 Least deprived           4,181,455 (73.7)   
                    Unknown                      822,855 (66.7)   
dementia            False                     24,675,140 (75.2)   
                    True                         141,945 (73.3)   
diabetes            False                     22,181,755 (74.5)   
                    True                       2,635,330 (81.2)   
hypertension        False                     23,192,570 (74.6)   
                    True                       1,624,515 (83.8)   
learning_disability False                     24,673,020 (75.1)   
                    True                         144,065 (81.9)   

                                                ethnicity new 5  \
group               subgroup                                      
all                 with records              24,762,715 (75.0)   
age_band            0-19                       3,993,025 (62.8)   
                    20-29                      3,056,200 (72.5)   
                    30-39                      4,002,260 (81.6)   
                    40-49                      3,273,850 (81.8)   
                    50-59                      3,210,730 (80.2)   
                    60-69                      2,614,625 (80.3)   
                    70-79                      2,257,210 (79.6)   
                    80+                        2,328,110 (69.3)   
                    missing                       26,710 (31.3)   
sex                 F                         12,715,185 (76.6)   
                    M                         12,047,530 (73.3)   
region              East                       4,368,735 (75.9)   
                    East Midlands              3,295,160 (77.0)   
                    London                     1,440,175 (80.3)   
                    North East                   900,925 (76.7)   
                    North West                 1,648,400 (76.7)   
                    South East                 1,229,985 (74.8)   
                    South West                 2,526,855 (74.2)   
                    West Midlands                817,725 (80.9)   
                    Yorkshire and The Humber   2,711,575 (76.2)   
imd                 1 Most depr

### Count of Missings

Around 5.8 million patients who have been registered in OpenSAFELY-TPP have no recorded etnicity across all three codelists. 2021-PRIMIS is the least well-populated with around 10 million patients having no 2021-PRIMIS recording of ethnicity.  

In [30]:
local_patient_counts(
         definitions,  output_path, missing= True
    )

ethnicity 5  \
group               subgroup                                     
all                 with records              8,201,825 (24.8)   
age_band            0-19                      2,349,970 (37.0)   
                    20-29                     1,149,790 (27.3)   
                    30-39                       891,710 (18.2)   
                    40-49                       721,205 (18.0)   
                    50-59                       787,705 (19.7)   
                    60-69                       636,920 (19.6)   
                    70-79                       575,145 (20.3)   
                    80+                       1,030,935 (30.7)   
                    missing                      58,445 (68.5)   
sex                 F                         3,852,110 (23.2)   
                    M                         4,349,715 (26.5)   
region              East                      1,374,695 (23.9)   
                    East Midlands               976,300 (22.8)   
                    London                      349,115 (19.5)   
                    North East                  272,210 (23.2)   
                    North West                  497,575 (23.2)   
                    South East                  411,670 (25.0)   
                    South West                  873,985 (25.7)   
                    West Midlands               191,415 (18.9)   
                    Yorkshire and The Humber    841,435 (23.7)   
imd                 1 Most deprived           1,512,995 (23.3)   
                    2                         1,545,200 (23.8)   
                    3                         1,629,695 (24.2)   
                    4                         1,610,155 (25.2)   
                    5 Least deprived          1,493,325 (26.3)   
                    Unknown                     410,465 (33.3)   
dementia            False                     8,150,045 (24.8)   
                    True                         51,780 (26.7)   
diabetes            False                     7,593,255 (25.5)   
                    True                        608,575 (18.8)   
hypertension        False                     7,887,935 (25.4)   
                    True                        313,895 (16.2)   
learning_disability False                     8,170,020 (24.9)   
                    True                         31,810 (18.1)   

                                               ethnicity new 5  \
group               subgroup                                     
all                 with records              8,256,195 (25.0)   
age_band            0-19                      2,363,900 (37.2)   
                    20-29                     1,158,745 (27.5)   
                    30-39                       903,505 (18.4)   
                    40-49                       729,850 (18.2)   
                    50-59                       792,480 (19.8)   
                    60-69                       639,785 (19.7)   
                    70-79                       576,920 (20.4)   
                    80+                       1,032,385 (30.7)   
                    missing                      58,620 (68.7)   
sex                 F                         3,878,315 (23.4)   
                    M                         4,377,880 (26.7)   
region              East                      1,384,665 (24.1)   
                    East Midlands               982,485 (23.0)   
                    London                      354,395 (19.7)   
                    North East                  273,180 (23.3)   
                    North West                  500,345 (23.3)   
                    South East                  414,000 (25.2)   
                    South West                  876,995 (25.8)   
                    West Midlands               193,340 (19.1)   
                    Yorkshire and The Humber    845,095 (23.8)   
imd                 1 Most deprived           1,529,575 (23.5)   
                    2   

### Count by Category

The 2022-SNOMED codelist is most well-populated for White (15.9 million), Mixed(360,000), Asian(1.7 million) and Black (570,000) ethnicities. The 2020-CTV3 codelist classifies more people as other than the SNOMED codelist (550,000 and 470,000 respectively), however, the 2020-CTV3 codelist included codes some relating to religion rather than ethnicity (e.g. “XaJSe: Muslim - ethnic category 2001 census”) which were excluded from the 2022-SNOMED codelist.

In [31]:
local_patient_counts(
         definitions,  output_path,code_dict, categories=True,missing=False
    )

White ethnicity 5  \
group               subgroup                                      
all                 with records              20,501,205 (82.8)   
age_band            0-19                       3,070,020 (76.9)   
                    20-29                      2,308,045 (75.5)   
                    30-39                      3,049,690 (76.2)   
                    40-49                      2,541,775 (77.6)   
                    50-59                      2,785,445 (86.8)   
                    60-69                      2,369,120 (90.6)   
                    70-79                      2,132,730 (94.5)   
                    80+                        2,223,270 (95.5)   
                    missing                       21,110 (79.0)   
sex                 F                         10,588,940 (83.3)   
                    M                          9,912,265 (82.3)   
region              East                       3,748,930 (85.8)   
                    East Midlands              2,757,375 (83.7)   
                    London                       762,495 (52.9)   
                    North East                   799,045 (88.7)   
                    North West                 1,527,660 (92.7)   
                    South East                 1,094,080 (89.0)   
                    South West                 2,367,015 (93.7)   
                    West Midlands                528,045 (64.6)   
                    Yorkshire and The Humber   2,271,135 (83.8)   
imd                 1 Most deprived            3,684,080 (74.1)   
                    2                          3,857,340 (78.2)   
                    3                          4,319,800 (84.8)   
                    4                          4,236,845 (88.8)   
                    5 Least deprived           3,798,605 (91.0)   
                    Unknown                      604,535 (73.7)   
dementia            False                     20,365,485 (82.7)   
                    True                         135,720 (95.7)   
diabetes            False                     18,363,150 (83.0)   
                    True                       2,138,055 (81.2)   
hypertension        False                     19,019,450 (82.2)   
                    True                       1,481,755 (91.3)   
learning_disability False                     20,373,490 (82.8)   
                    True                         127,715 (88.7)   

                                             White ethnicity new 5  \
group               subgroup                                         
all                 with records                 20,514,355 (82.8)   
age_band            0-19                          3,071,530 (76.9)   
                    20-29                         2,310,130 (75.6)   
                    30-39                         3,054,410 (76.3)   
                    40-49                         2,544,920 (77.7)   
                    50-59                         2,786,690 (86.8)   
                    60-69                         2,369,565 (90.6)   
                    70-79                         2,132,735 (94.5)   
                    80+                           2,223,270 (95.5)   
                    missing                          21,100 (79.0)   
sex                 F                            10,595,975 (83.3)   
                    M                             9,918,375 (82.3)   
region              East                          3,751,915 (85.9)   
                    East Midlands                 2,758,360 (83.7)   
                    London                          764,615 (53.1)   
                    North East                      799,300 (88.7)   
                    North West                    1,527,455 (92.7)   
                    South East                    1,094,795 (89.0)   
                    South West                    2,368,080 (93.7)   
                    West Midlands                   528,120 (64.6)   
                    Yorksh

### Latest vs. Most Common

Overall 98% of the latest 6 group ethnicity matched the most frequent 6 group ethnicity. 99.3% (2022-SNOMED) of those with the most recent ethnicity ‘White’ also had the most frequent ethnicity ‘White’. Mixed was the least concordant with 77.0% of those with the most recent ethnicity ‘Mixed’ also had the most frequent ethnicity ‘Mixed’. 3.1% of those with latest ethnicity ‘Black’ also had the most frequent ethnicity ‘White’.


In [32]:
for definition in definitions:
    for suffix in suffixes:
        df_sum = pd.read_csv(f'../output/{output_path}/simple_latest_common_{definition}{suffix}_fullset.csv').set_index(definition)
        # sort rows by category index
        df_sum.columns = df_sum.columns.str.replace(definition + "_", "")
        df_sum.columns = df_sum.columns.str.lower()
        df_sum = df_sum.reindex(list(code_dict[definition].values()))
        
        df_counts = pd.DataFrame(
            np.diagonal(df_sum),
            index=df_sum.index,
        #   columns=[f"matching (n={np.diagonal(df_sum).sum()})"],
        )

        df_sum2 = df_sum.copy(deep=True)
        np.fill_diagonal(df_sum2.values, 0)
        df_diag = pd.DataFrame(
            df_sum2.sum(axis=1),
        )
        df_out = df_counts.merge(df_diag, right_index=True, left_index=True)
        columns=round(df_out.sum()/df_out.sum(axis=1).sum()*100,1)
        df_out.columns=[f"matching ({columns[0]}%)",f"not matching ({columns[1]}%)"]
        display(df_out)
        
        if code_dict != "":
            lowerlist = [x.lower() for x in (list(code_dict[definition].values()))]
            df_sum = df_sum[lowerlist]
        else:
            df_sum = df_sum.reindex(sorted(df_sum.columns), axis=1)

        # Combine count and percentage columns
        df_sum["population"]=df_sum.sum(axis = 1)
        for item in lowerlist:
            df_sum[item + "_pct"]= round(
                    (df_sum[item].div(df_sum["population"])) * 100, 1
                )
        
            df_sum[item] = (
                    df_sum[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_sum[item + "_pct"].astype(str)
                    + ")"
                )
        df_sum = df_sum[lowerlist]

        display(df_sum)
    # df_expanded = pd.read_csv(f'../output/{output_path}/tables/latest_common_expanded_{definition}.csv').set_index(definition)
    
    # display(df_expanded)


,matching (97.7%),not matching (2.3%)
ethnicity_5,,
White,"20,463,590","157,970"
Mixed,"438,935","127,000"
Asian,"2,174,830","88,910"
Black,"777,160","65,380"
Other,"803,000","150,885"


,white,mixed,asian,black,other
ethnicity_5,,,,,
White,"20,463,590 (99.2)","43,385 (0.2)","28,555 (0.1)","21,435 (0.1)","64,595 (0.3)"
Mixed,"57,240 (10.1)","438,935 (77.6)","18,085 (3.2)","33,480 (5.9)","18,195 (3.2)"
Asian,"25,760 (1.1)","15,350 (0.7)","2,174,830 (96.1)","5,585 (0.2)","42,215 (1.9)"
Black,"23,870 (2.8)","25,825 (3.1)","5,405 (0.6)","777,160 (92.2)","10,280 (1.2)"
Other,"80,740 (8.5)","16,990 (1.8)","42,195 (4.4)","10,960 (1.1)","803,000 (84.2)"


,matching (97.0%),not matching (3.0%)
ethnicity_5,,
White,"1,322,775","14,285"
Mixed,"23,485","10,930"
Asian,"171,805","7,490"
Black,"53,870","5,805"
Other,"36,030","11,890"


,white,mixed,asian,black,other
ethnicity_5,,,,,
White,"1,322,775 (98.9)","3,975 (0.3)","2,720 (0.2)","1,810 (0.1)","5,780 (0.4)"
Mixed,"5,205 (15.1)","23,485 (68.2)","1,685 (4.9)","2,840 (8.3)","1,200 (3.5)"
Asian,"2,850 (1.6)","1,055 (0.6)","171,805 (95.8)",655 (0.4),"2,930 (1.6)"
Black,"2,590 (4.3)","1,705 (2.9)",620 (1.0),"53,870 (90.3)",890 (1.5)
Other,"5,910 (12.3)","1,165 (2.4)","3,990 (8.3)",825 (1.7),"36,030 (75.2)"


,matching (97.9%),not matching (2.1%)
ethnicity_new_5,,
White,"20,482,790","145,785"
Mixed,"443,835","126,780"
Asian,"2,224,215","80,785"
Black,"781,205","63,610"
Other,"694,955","118,290"


...

,white,mixed,asian,black,other
ethnicity_new_5,,,,,
White,"1,319,570 (99.3)","3,875 (0.3)","2,710 (0.2)","1,805 (0.1)",595 (0.0)
Mixed,"5,240 (15.7)","23,245 (69.6)","1,705 (5.1)","2,850 (8.5)",345 (1.0)
Asian,"2,890 (1.6)","1,055 (0.6)","172,145 (96.9)",655 (0.4),900 (0.5)
Black,"2,590 (4.4)","1,630 (2.8)",620 (1.1),"53,580 (91.6)",105 (0.2)
Other,"4,875 (16.0)",910 (3.0),"2,945 (9.7)",655 (2.2),"21,045 (69.2)"


,matching (98.0%),not matching (2.0%)
ethnicity_primis_5,,
White,"15,694,085","111,210"
Mixed,"408,965","91,330"
Asian,"1,888,365","56,565"
Black,"606,890","46,270"
Other,"664,325","89,120"


,white,mixed,asian,black,other
ethnicity_primis_5,,,,,
White,"15,694,085 (99.3)","32,390 (0.2)","21,965 (0.1)","14,980 (0.1)","41,875 (0.3)"
Mixed,"41,995 (8.4)","408,965 (81.7)","13,965 (2.8)","23,215 (4.6)","12,155 (2.4)"
Asian,"20,335 (1.0)","11,315 (0.6)","1,888,365 (97.1)","3,360 (0.2)","21,555 (1.1)"
Black,"17,160 (2.6)","19,460 (3.0)","3,395 (0.5)","606,890 (92.9)","6,255 (1.0)"
Other,"46,635 (6.2)","11,600 (1.5)","24,065 (3.2)","6,820 (0.9)","664,325 (88.2)"


,matching (97.8%),not matching (2.2%)
ethnicity_primis_5,,
White,"1,194,405","7,625"
Mixed,"24,370","8,270"
Asian,"168,905","4,260"
Black,"49,850","3,755"
Other,"4,380","8,860"


,white,mixed,asian,black,other
ethnicity_primis_5,,,,,
White,"1,194,405 (99.4)","3,330 (0.3)","2,395 (0.2)","1,570 (0.1)",330 (0.0)
Mixed,"4,220 (12.9)","24,370 (74.7)","1,385 (4.2)","2,520 (7.7)",145 (0.4)
Asian,"2,525 (1.5)",890 (0.5),"168,905 (97.5)",560 (0.3),285 (0.2)
Black,"1,985 (3.7)","1,250 (2.3)",440 (0.8),"49,850 (93.0)",80 (0.1)
Other,"4,630 (35.0)",860 (6.5),"2,690 (20.3)",680 (5.1),"4,380 (33.1)"


### State Change

Patients whose latest recorded ethnicity were categorised as Mixed were most likely to have a discordant ethnicity recording (32%) with 16.7% of the 358,055 patients with the latest recording of Mixed ethnicity also having a recording of White ethnicity. Surprisingly 5.5% of those with the latest recorded ethnicity categorised as Black were also had a recorded ethnicity of White 

In [33]:
for definition in definitions:
    for suffix in suffixes:
        df_state_change = pd.read_csv(f'../output/{output_path}/simple_state_change_{definition}{suffix}_fullset.csv').set_index(definition)
        df_state_change.columns = df_state_change.columns.str.replace(definition + "_", "")
        #resort rows
        df_state_change = df_state_change.reindex(list(code_dict[definition].values()))
        df_state_change = df_state_change.reset_index()
        
        df_state_change[definition]=df_state_change[definition]+": " +df_state_change["n"].apply(lambda x: "{:,.0f}".format(x))
        df_state_change = df_state_change.set_index(definition)
        for item in lowerlist:
            df_state_change[item + "_pct"]= round(
                    (df_state_change[item].div(df_state_change["n"])) * 100, 1
                )
        
            df_state_change[item] = (
                    df_state_change[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_state_change[item + "_pct"].astype(str)
                    + ")"
                )
        df_state_change=df_state_change[lowerlist]
        display(df_state_change)

,white,mixed,asian,black,other
ethnicity_5,,,,,
"White: 20,501,205","20,501,205 (100.0)","75,805 (0.4)","38,020 (0.2)","31,425 (0.2)","127,420 (0.6)"
"Mixed: 479,120","75,745 (15.8)","479,120 (100.0)","22,315 (4.7)","42,155 (8.8)","25,030 (5.2)"
"Asian: 2,194,055","49,810 (2.3)","30,605 (1.4)","2,194,055 (100.0)","10,490 (0.5)","70,045 (3.2)"
"Black: 792,185","39,705 (5.0)","50,615 (6.4)","8,250 (1.0)","792,185 (100.0)","17,915 (2.3)"
"Other: 850,520","96,315 (11.3)","26,725 (3.1)","57,595 (6.8)","14,680 (1.7)","850,520 (100.0)"


,white,mixed,asian,black,other
ethnicity_5,,,,,
"White: 1,331,970","1,331,970 (100.0)","8,720 (0.7)","4,635 (0.3)","3,685 (0.3)","15,540 (1.2)"
"Mixed: 31,705","7,735 (24.4)","31,705 (100.0)","2,360 (7.4)","4,095 (12.9)","2,335 (7.4)"
"Asian: 176,690","6,430 (3.6)","3,610 (2.0)","176,690 (100.0)","1,655 (0.9)","7,535 (4.3)"
"Black: 57,740","4,700 (8.1)","4,950 (8.6)","1,155 (2.0)","57,740 (100.0)","2,070 (3.6)"
"Other: 45,400","8,145 (17.9)","2,350 (5.2)","5,695 (12.5)","1,420 (3.1)","45,400 (100.0)"


,white,mixed,asian,black,other
ethnicity_new_5,,,,,
"White: 20,514,355","20,514,355 (100.0)","76,245 (0.4)","38,795 (0.2)","31,500 (0.2)","102,725 (0.5)"
"Mixed: 483,350","76,310 (15.8)","483,350 (100.0)","23,150 (4.8)","42,300 (8.8)","20,250 (4.2)"
"Asian: 2,239,570","51,035 (2.3)","31,840 (1.4)","2,239,570 (100.0)","10,655 (0.5)","52,180 (2.3)"
"Black: 794,645","39,780 (5.0)","50,760 (6.4)","8,365 (1.1)","794,645 (100.0)","14,650 (1.8)"
"Other: 730,800","75,770 (10.4)","20,675 (2.8)","41,295 (5.7)","11,835 (1.6)","730,800 (100.0)"


,white,mixed,asian,black,other
ethnicity_new_5,,,,,
"White: 1,325,605","1,325,605 (100.0)","8,560 (0.6)","4,575 (0.3)","3,605 (0.3)","7,105 (0.5)"
"Mixed: 31,040","7,610 (24.5)","31,040 (100.0)","2,340 (7.5)","4,040 (13.0)","1,190 (3.8)"
"Asian: 175,665","6,385 (3.6)","3,575 (2.0)","175,665 (100.0)","1,625 (0.9)","4,125 (2.3)"
"Black: 56,900","4,645 (8.2)","4,875 (8.6)","1,125 (2.0)","56,900 (100.0)",885 (1.6)
"Other: 28,600","5,605 (19.6)","1,570 (5.5)","3,820 (13.4)",900 (3.1),"28,600 (100.0)"


,white,mixed,asian,black,other
ethnicity_primis_5,,,,,
"White: 15,715,345","15,715,345 (100.0)","50,385 (0.3)","27,765 (0.2)","19,970 (0.1)","72,300 (0.5)"
"Mixed: 433,665","53,200 (12.3)","433,665 (100.0)","16,460 (3.8)","27,850 (6.4)","15,420 (3.6)"
"Asian: 1,898,245","35,540 (1.9)","20,000 (1.1)","1,898,245 (100.0)","5,435 (0.3)","32,365 (1.7)"
"Black: 616,020","25,215 (4.1)","32,715 (5.3)","4,895 (0.8)","616,020 (100.0)","9,825 (1.6)"
"Other: 688,810","54,270 (7.9)","16,450 (2.4)","31,025 (4.5)","8,700 (1.3)","688,810 (100.0)"


,white,mixed,asian,black,other
ethnicity_primis_5,,,,,
"White: 1,199,425","1,199,425 (100.0)","6,275 (0.5)","3,730 (0.3)","2,605 (0.2)","5,325 (0.4)"
"Mixed: 30,370","5,980 (19.7)","30,370 (100.0)","1,845 (6.1)","3,210 (10.6)",710 (2.3)
"Asian: 171,540","4,965 (2.9)","2,505 (1.5)","171,540 (100.0)","1,070 (0.6)","2,265 (1.3)"
"Black: 52,420","3,275 (6.2)","3,365 (6.4)",770 (1.5),"52,420 (100.0)",620 (1.2)
"Other: 10,865","5,060 (46.6)","1,230 (11.3)","3,085 (28.4)",850 (7.8),"10,865 (100.0)"
